In [39]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import plotly.graph_objs as go
import pandas as pd

In [40]:
# Load the iris dataset as a CSV file
data = np.loadtxt('iris.csv', delimiter=',', skiprows=1, usecols=(0, 1, 2, 3))
target = np.loadtxt('iris.csv', delimiter=',', skiprows=1, usecols=(4,), dtype=float)

# Convert the target labels to numerical values
target[target == 'Iris-setosa'] = '0'
target[target == 'Iris-versicolor'] = '1'
target[target == 'Iris-virginica'] = '2'

# Scale the features manually
mean = np.mean(data, axis=0)
std = np.std(data, axis=0)
data_scaled = (data - mean) / std

# Split the data into training and testing sets
np.random.seed(42)
indices = np.random.permutation(len(data_scaled))
train_indices, test_indices = indices[:int(0.5*len(data_scaled))], indices[int(0.5*len(data_scaled)):]
X_train, y_train = data_scaled[train_indices], target[train_indices]
X_test, y_test = data_scaled[test_indices], target[test_indices]


<ipython-input-40-037544a62219>:6: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

<ipython-input-40-037544a62219>:7: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison

<ipython-input-40-037544a62219>:8: FutureWarning:

elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison



In [41]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()

In [42]:
# Define the logistic regression model using PyTorch's nn.Module class
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, x):
        out = self.linear(x)
        return out

# Define the neural network model using PyTorch's nn.Module class
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out

# Define the models and the optimizer
lr_model = LogisticRegression(input_size=4, num_classes=3)
nn_model = NeuralNetwork(input_size=4, hidden_size=64, num_classes=3)
lr_optimizer = optim.SGD(lr_model.parameters(), lr=0.01)
nn_optimizer = optim.SGD(nn_model.parameters(), lr=0.01)

In [43]:
from collections import defaultdict

lr_history = defaultdict(list)
nn_history = defaultdict(list)

In [44]:
# Train the logistic regression model
for epoch in range(1000):
    # Forward pass
    logits = lr_model(X_train)
    loss = nn.functional.cross_entropy(logits, y_train)

    # Backward pass
    lr_optimizer.zero_grad()
    loss.backward()
    lr_optimizer.step()

    # Compute training accuracy
    preds = torch.argmax(logits, axis=1)
    accuracy = torch.mean((preds == y_train).float())

    # Save loss and accuracy for plotting
    lr_history['loss'].append(loss.item())
    lr_history['accuracy'].append(accuracy.item())

# Train the neural network model
for epoch in range(1000):
    # Forward pass
    logits = nn_model(X_train)
    loss = nn.functional.cross_entropy(logits, y_train)

    # Backward pass
    nn_optimizer.zero_grad()
    loss.backward()
    nn_optimizer.step()

    # Compute training accuracy
    preds = torch.argmax(logits, axis=1)
    accuracy = torch.mean((preds == y_train).float())

    # Save loss and accuracy for plotting
    nn_history['loss'].append(loss.item())
    nn_history['accuracy'].append(accuracy.item())

In [45]:
# Smooth the curves using a moving average over 5 points
window_size = 5
weights = np.repeat(1.0, window_size)/window_size
lr_loss_smooth = np.convolve(lr_history['loss'], weights, 'valid')
nn_loss_smooth = np.convolve(nn_history['loss'], weights, 'valid')
lr_acc_smooth = np.convolve(lr_history['accuracy'], weights, 'valid')
nn_acc_smooth = np.convolve(nn_history['accuracy'], weights, 'valid')

# Define the data traces for the loss plot
trace1 = go.Scatter(x=np.arange(window_size//2, len(lr_loss_smooth)+window_size//2), y=lr_loss_smooth, name='Logistic Regression', mode='lines+markers')
trace2 =  go.Scatter(x=np.arange(window_size//2, len(nn_loss_smooth)+window_size//2), y=nn_loss_smooth, name='Neural Network', mode='lines+markers')
data_loss = [trace1, trace2]

# Define the layout for the loss plot
layout_loss = go.Layout(title='Training Loss over Epochs', xaxis=dict(title='Epoch'), yaxis=dict(title='Loss', type='log'))

# Define the data traces for the accuracy plot
trace1 = go.Scatter(x=np.arange(window_size//2, len(lr_acc_smooth)+window_size//2), y=lr_acc_smooth, name='Logistic Regression', mode='lines+markers')
trace2 = go.Scatter(x=np.arange(window_size//2, len(nn_acc_smooth)+window_size//2), y=nn_acc_smooth, name='Neural Network', mode='lines+markers')
data_accuracy = [trace1, trace2]

# Define the layout for the accuracy plot
layout_accuracy = go.Layout(title='Training Accuracy over Epochs', xaxis=dict(title='Epoch'), yaxis=dict(title='Accuracy', tickformat=".2%"))

# Create the loss and accuracy plots
fig_loss = go.Figure(data=data_loss, layout=layout_loss)
fig_accuracy = go.Figure(data=data_accuracy, layout=layout_accuracy)

# Add a legend to the plots
fig_loss.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))
fig_accuracy.update_layout(legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01))

# Show the plots
fig_loss.show()
fig_accuracy.show()

